# Week 01


## After-class report

We started exploring recurrence relations in divide and conquer (D&C) algorithms. These techniques are covered in [chapter 1](https://jeffe.cs.illinois.edu/teaching/algorithms/book/01-recursion.pdf) of Jeff Erickson's book.

A familiar example of a D&C algorithm is _mergesort._ It delivers results in about $n\log_2 n$ steps, for an array of $n$ elements, which is far better than brute force sorting (such as selection sort) that requires $n^2$ steps.

Mergesort is based on a simple and fast way to merge two arrays that are already sorted.


In [ ]:
def merge_sorted_arrays(arr1, arr2):
    """
    Merge two sorted arrays into a single sorted array. The method processes
    both arrays from left to right, comparing their elements and appending
    the smaller one to the result array.

    Args:
        arr1: First sorted array
        arr2: Second sorted array

    Returns:
        A new sorted array containing all elements from both input arrays
    """
    merged = []  # Resultant merged array
    i, j = 0, 0  # Leftmost pointers for arr1 and arr2

    # Compare elements from both arrays and add the smaller one
    while i < len(arr1) and j < len(arr2):
        if arr1[i] <= arr2[j]:
            merged.append(arr1[i])
            i += 1  # advance leftmost pointer
        else:
            merged.append(arr2[j])
            j += 1  # advance leftmost pointer

    # Add remaining elements from arr1, if any
    while i < len(arr1):
        merged.append(arr1[i])
        i += 1

    # Add remaining elements from arr2, if any
    while j < len(arr2):
        merged.append(arr2[j])
        j += 1

    return merged

`merge` above works in linear time, i.e., it requires $n$ steps to merge two arrays with $n$ total elements.

Given an array to sort, for example:


In [ ]:
data = [5, 3, 8, 6, 2, 7, 4, 1]

the idea is to break it down to multiple arrays with one element each:


In [ ]:
a = [5]
b = [3]
c = [8]
d = [6]
e = [2]
f = [7]
g = [4]
h = [1]

These arrays are, by definition, sorted. And we can feed pairs of them to `merge`.


In [ ]:
ab = merge_sorted_arrays(a, b)
cd = merge_sorted_arrays(c, d)
ef = merge_sorted_arrays(e, f)
gh = merge_sorted_arrays(g, h)
print(ab, cd, ef, gh)  # Output: [3, 5] [6, 8] [2, 7] [1, 4]

This results to sorted arrays with two elements each that can be fed back to merge.


In [ ]:
abcd = merge_sorted_arrays(ab, cd)
efgh = merge_sorted_arrays(ef, gh)
print(abcd, efgh)  # Output: [3, 5, 6, 8] [1, 2, 4, 7]

And finally we can merge the last two sorted arrays, with four elements each:


In [ ]:
abcdefgh = merge_sorted_arrays(abcd, efgh)
print(abcdefgh)  # Output: [1, 2, 3, 4

If we look at the large `merge` call:

```python
abcdefgh = merge_sorted_arrays(abcd, efgh)
```

we can replace its arguments `abcd` and `efgh` with the `merge` calls they produced them, and the replace their arguments with the corresponding `merge` calls, etc, as shown below.


In [ ]:
# fmt: off

abcdefgh = merge_sorted_arrays(     # level 1
    merge_sorted_arrays(            #   level 2
        merge_sorted_arrays(a, b),  #     level 3
        merge_sorted_arrays(c, d)), #     level 3
    merge_sorted_arrays(            #   level 2
        merge_sorted_arrays(e, f),  #     level 3
        merge_sorted_arrays(g, h)), #     level 3
)

print(abcdefgh)

In the code above, the first call to `merge`, labeled *level 1,* has to wait for the two other calls it makes to `merge`, labeled *level 2.* Each of these calls also makes calls *(level 3)* that need to be completed first. And that's how recursion works: the first call has to wait for the second call to complete, the second for the first, etc. The last calls execute first, in other words, Last-In/First-out: that's a stack and that's how recursion works.

## Performance analysis

How fast does mergesort work? We start with an obvious observation: the time it takes to mergesort an array of size $n$ is twice the time to sort two smaller arrays of half size:

$$
T(n) = 2 T(n/2) +f(n)
$$

Here, $T(n)$ is the time is takes to mergesort an array of size $n$. And $f(n)$ is the time it takes to assemble two partial solutions of size $n/2$ to a sorted array with $n$ elements. From a simple review of method `merge` above, that assembles two sorted arrays into an also sorted array, $f(n) \approx n$.

For an array with $n=8$ we can write:

$$ T(8) = 2T(4) + f(8) $$

Then $T(4) = 2T(2)+f(4)$; and $T(2)=2T(1)+f(2)$; and finally $T(1) = f(1)$. Using this last finding, we can solve backwards:


$$
\begin{align*} 
T(2) &= 2T(1)+f(2) \\ &= 2f(1) + f(2) \\ \\
T(4) &= 2T(2)+f(4) \\ &= 2(2f(1) + f(2)) + f(4) \\ &= 4f(1)+2f(2)+f(4) \\ \\
T(8) &= 2T(4)+f(8) \\ &= 2(4f(1)+2f(2)+f(4)) + f(8) \\
     &= 8f(1) + 4f(2) +2f(4) + f(8)
\end{align*}
$$

Given that $f(n)\approx n$, the last equation can be rewritten as:

$$
\begin{align*}
T(8) & \approx 8\times 1 + 4\times 2 +8 \\ &= 24 = 8\times 3 \\ &= 8\log_2 8
\end{align*}
$$

So, empirically we've shown that for mergesort, $T(n) \approx n\log_2 n$ or (to use better notation) $T(n)\in\mathcal O(n\log_2 n)$. Now, of course we have to prove that this is the case for any size array, not for the special case $n=8$ we analyzed above.